## Number of energy consumption
For each gaussian:
- `preprocessCUDA`: 151 MACs
For each elem:
- `radix sort`: 24 MACs
For each pixel:
- `render`: 3 + r * (1 + 256 * 14) + 2 * 3 = 9 + r * 3585
## Basic Information
width: 1200
height: 680

Mapping step 5:
tile.x, tile.y = 75, 43
num_sort = 23463167
P = 867364
mean rounds: 2.9
visible gaussians = 833384

## Metrics
Original for frames 30:

- Final Average ATE RMSE: 0.04 cm
- Average PSNR: 40.63
- Average Depth RMSE: 0.18 cm
- Average Depth L1: 0.18 cm
- Average MS-SSIM: 0.996
- Average LPIPS: 0.024

In [ ]:
tile_x, tile_y = 75, 43
block_size = 256
e = 0.5 # pj/MAC

# Mapping step 5:
sort_size = 23463167
P = 867364
visible_gaussians = 833384
mean_rounds = 2.9

preprocess = 151 * visible_gaussians
radix_sort = 24 * sort_size
render =  (3 + mean_rounds * (1 + block_size * 14) + 2 * 3) * block_size * tile_x * tile_y

print(f"Preprocess: {preprocess * e / 1e6:.2f} muj")
print(f"Radix sort: {radix_sort * e / 1e6:.2f} muj")
print(f"Render: {render * e / 1e6:.2f} muj")
print(f"Total: {(preprocess + radix_sort + render) * e / 1e6:.2f} muj")


# Test image descriptor match algorithms

In [1]:
import os
import re
import pandas as pd

log_dir = "/home/xthuang/code/slam/SplaTAM-Maxime/logs"
psnr_pattern = r'Average PSNR: (\d+\.\d+)'
ATE_rmse_pattern = r'Final Average ATE RMSE: (\d+\.\d+) cm'

res = {}
scenes = ["room0", "room1", "room2",
        "office0", "office1", "office2",
        "office_3", "office4"]
rows = set()

for file in os.listdir(log_dir):
    log_file = os.path.join(log_dir, file)
    splits = file.split('_')
    dataset_id = int(splits[-1][:-4])
    dataset = scenes[dataset_id]
    method = '_'.join(splits[:-1])
    
    with open(log_file, 'r') as f:
        for line in f.readlines()[-10:]:
            psnr_match = re.search(psnr_pattern, line)
            if psnr_match:
                psnr = float(psnr_match.group(1))
            
            ATE_rmse_match = re.search(ATE_rmse_pattern, line)
            if ATE_rmse_match:
                ATE_rmse = float(ATE_rmse_match.group(1))
    
    if dataset+'_rmse' not in res:
        res[dataset+"_psnr"] = {}
        res[dataset+"_rmse"] = {}
    res[dataset+"_rmse"][method] = ATE_rmse
    res[dataset+"_psnr"][method] = psnr
    
    rows.add(method)
df = pd.DataFrame(index=list(rows), columns=sorted(res.keys()))
for col in res:
    for row in res[col]:
        df.loc[row, col] = res[col][row]
        
df.to_csv("output.csv")
    